In [ ]:
!pip install ahocorapy

## Bibliotecas

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils

In [2]:
import urllib3
import pandas as pd
import numpy as np

from time import sleep
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from ahocorapy.keywordtree import KeywordTree

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-web-security")
options.add_argument("--headless")
prefs = {"download_restrictions": 3}
options.add_experimental_option("prefs", prefs)

## Definição das labels e dos templates

In [3]:
labels = ['licitacoes', 'contratos', 'pagamentos', 'despesas', 'pessoal', 'receitas']
NUMBER_OF_LABELS = len(labels)

In [4]:
label_to_int = lambda x: {"licitacoes" : 0, "contratos" : 1, "pagamentos" : 2, "despesas" : 3, 
                          "pessoal" : 4, "receitas" : 5}[x]

In [5]:
int_to_label = lambda x: {0 : "licitacoes", 1 : "contratos", 2 : "pagamentos", 3 : "despesas", 
                          4 : "pessoal", 5 : "receitas"}[x]

In [6]:
templates = ['Template 2',
       'Portal TP',
       'Siplanweb',
       'Betha',
       'Sintese e Tecnologia',
       'ABO',
       'PT',
       'ADPM',
       'Municipal Net',
       'Portal Facil (60)',
       'Portal Facil (46)',
       'Memory',
       'GRP',
       'Template 1 (22)',
       'Template 1 (9)']

## Criação das Keyword Trees

In [7]:
kwtree_url = KeywordTree(case_insensitive=True)

tags_url = ["receita", "licita", "empenh", "paga", "relat", "lei", "contrat", "servidor", "folha", "obra", 
            "repass", "conta", "concurs", "orcamen", "conselh", "diaria", "legisla"]

for tag in tags_url:
    kwtree_url.add(tag)
kwtree_url.finalize()

In [8]:
kwtree_page = KeywordTree(case_insensitive=True)


tags_page = ["Transparência", "12.527/2011", "45.969/2012", "www.transparencia.mg.gov.br", "Lei de Acesso à Informação", 
             "Conselho", "CODEMA", "CMDCA", "F.A.Q", "FAQ", "Perguntas Frequentes", "Pedidos", "Estrutura Organizacional", 
             "Endereço", "Telefone", "Horário de Atendimento", "Conselhos Municipais", "Receitas", "Consolidado", "Liquida", 
             "Previs", "Arrecada", "Classificação","Balanço", "Contas", "Meta", "Plano de Metas", "Gestão fiscal", 
             "Resumido de Execução", "Diretrizes", "Orçament",  "Pagamentos", "Pagar", "Empenhos", "N° de empenho", 
             "N° do empenho", "Valor", "Favorecido", "Licitaç", "Status", "Modalidade", "Resultado", "Status","Contrato", 
             "Vigência", "Terceiro Setor", "Parcerias", "Repasses", "Data de celebração", "Data", "Conveniado", "Origem", 
             "Recurso", "Concurso", "Conselho", "Pensão", "Plano Plurianual", "Concurso Público", "Obra", "Situação", 
             "Servidor", "Nome", "Cargo", "Função", "Relatorio Mensal", "Despesa com Pessoal", "Despesas com pessoal", 
             "Aposentado", "Pensionista", "Diária", "Viagen", "Viagem", "Periodo", "Convênios", "Destino", "Motivo", 
             "Prestação", "Repasse", "Legisl"]

for tag in tags_page:
    kwtree_page.add(tag)
kwtree_page.finalize()

## Definição das features de entrada do modelo

In [9]:
#vector = templates + tags_url + tags_page
vector = tags_url + tags_page
NUMBER_OF_ATRIBUTES = len(vector)

In [10]:
print(vector,"\n", NUMBER_OF_ATRIBUTES)

['receita', 'licita', 'empenh', 'paga', 'relat', 'lei', 'contrat', 'servidor', 'folha', 'obra', 'repass', 'conta', 'concurs', 'orcamen', 'conselh', 'diaria', 'legisla', 'Transparência', '12.527/2011', '45.969/2012', 'www.transparencia.mg.gov.br', 'Lei de Acesso à Informação', 'Conselho', 'CODEMA', 'CMDCA', 'F.A.Q', 'FAQ', 'Perguntas Frequentes', 'Pedidos', 'Estrutura Organizacional', 'Endereço', 'Telefone', 'Horário de Atendimento', 'Conselhos Municipais', 'Receitas', 'Consolidado', 'Liquida', 'Previs', 'Arrecada', 'Classificação', 'Balanço', 'Contas', 'Meta', 'Plano de Metas', 'Gestão fiscal', 'Resumido de Execução', 'Diretrizes', 'Orçament', 'Pagamentos', 'Pagar', 'Empenhos', 'N° de empenho', 'N° do empenho', 'Valor', 'Favorecido', 'Licitaç', 'Status', 'Modalidade', 'Resultado', 'Status', 'Contrato', 'Vigência', 'Terceiro Setor', 'Parcerias', 'Repasses', 'Data de celebração', 'Data', 'Conveniado', 'Origem', 'Recurso', 'Concurso', 'Conselho', 'Pensão', 'Plano Plurianual', 'Concurso Pú

## Funções para extração de features

In [14]:
def findPatterns(url, text, template):

    vector_results = [0] * len(vector)
    
    #determinando o template
    #idx = vector.index(template)
    #vector_results[idx] = 1
        
    #resultados do texto da url
    results_url = kwtree_url.search_all(url)
    results_url = [result[0] for result in results_url]
    results_unique_url = set(results_url)
    
    for result in results_unique_url:
        idx = vector.index(result)
        num_results = results_url.count(result)
        vector_results[idx] = num_results*100
        
    #resultados do texto da pagina
    results_page = kwtree_page.search_all(text)
    results_page = [result[0] for result in results_page]
    results_unique_page = set(results_page)

    for result in results_unique_page:
        idx = vector.index(result)
        num_results = results_page.count(result)
        vector_results[idx] = num_results
        

    return vector_results

In [15]:
def getText(url):

    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    sleep(2)
    
    try:
        soup = BeautifulSoup(driver.page_source, "lxml")
        
        for tags in soup(['script', 'style']):
            tags.decompose
            
        driver.close()
        
        return ' '. join(soup.stripped_strings)
        
    except:
        print("Erro ao tentar abrir a pagina: " + url)

In [16]:
def preProcessing(url, template):
    
    text = getText(url)
    vector = findPatterns(url, text, template)
    
    return vector

In [ ]:
preProcessing("https://www.municipalnet.com.br/index/?apl=contas-publicas?uid=congonhal", "Municipal Net")

## Criação do dataset de treinamento

In [18]:
dataset = [['https://pm-aracitaba.publicacao.siplanweb.com.br/licitacoes/painel','licitacoes','Siplanweb'],
 ['https://pm-aracitaba.publicacao.siplanweb.com.br/licitacoes/pregao-presencial','licitacoes','Siplanweb'],
 ['https://pm-aracitaba.publicacao.siplanweb.com.br/contratos','contratos','Siplanweb'],
 ['https://pm-cruzilia.publicacao.siplanweb.com.br/contratos', 'contratos', 'Siplanweb'],
 ['https://pm-aracitaba.publicacao.siplanweb.com.br/restos-pagar', 'pagamentos', 'Siplanweb'],
 ['https://pm-cruzilia.publicacao.siplanweb.com.br/restos-pagar','pagamentos','Siplanweb'],
 ['https://pm-aracitaba.publicacao.siplanweb.com.br/contas-publicas/documento/54','despesas','Siplanweb'],
 ['https://pm-aracitaba.publicacao.siplanweb.com.br/contas-publicas/documento/30','despesas','Siplanweb'],
 ['https://pm-aracitaba.publicacao.siplanweb.com.br/pessoal','pessoal','Siplanweb'],
 ['https://pm-cruzilia.publicacao.siplanweb.com.br/pessoal','pessoal','Siplanweb'],
 ['https://pm-cruzilia.publicacao.siplanweb.com.br/contas-receitas','receitas','Siplanweb'],
 ['https://pm-blonga.publicacao.siplanweb.com.br/contas-receitas','receitas','Siplanweb'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/compra/FLicitacao.xhtml?Param=PmIbiai','licitacoes','Sintese e Tecnologia'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/compra/FLicitacao.xhtml?Param=Turmalina','licitacoes','Sintese e Tecnologia'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/compra/FContrato.xhtml?Param=Turmalina','contratos','Sintese e Tecnologia'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/compra/FContrato.xhtml?Param=PmIbiai','contratos','Sintese e Tecnologia'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/despesa/FPagamentosRealizados.xhtml?Param=PmIbiai','pagamentos','Sintese e Tecnologia'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/despesa/FPagamentosRealizados.xhtml?Param=Turmalina','pagamentos','Sintese e Tecnologia'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/pcontas/FPContas.xhtml?Param=PmIbiai','despesas','Sintese e Tecnologia'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/pcontas/FPContas.xhtml?Param=Turmalina','despesas', 'Sintese e Tecnologia'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/folha/FFolhaPagamento.xhtml?Param=PmIbiai','pessoal','Sintese e Tecnologia'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/folha/FFolhaPagamento.xhtml?Param=Turmalina','pessoal','Sintese e Tecnologia'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/receita/FReceitasPrevisao.xhtml?Param=PmIbiai','receitas',  'Sintese e Tecnologia'],
 ['http://cidadesmg.com.br/portaltransparencia/faces/user/receita/FReceitasArrecadacao.xhtml?Param=Turmalina','receitas','Sintese e Tecnologia'],
 ['http://transparencia.virginia.mg.gov.br/Licitacao', 'licitacoes', 'ABO'],
 ['http://esic.vespasiano.mg.gov.br/Licitacao', 'licitacoes', 'ABO'],
 ['http://transparencia.virginia.mg.gov.br/Contrato', 'contratos', 'ABO'],
 ['http://esic.vespasiano.mg.gov.br/Contrato', 'contratos', 'ABO'],
 ['http://esic.vespasiano.mg.gov.br/Despesa', 'despesas', 'ABO'],
 ['http://transparencia.virginia.mg.gov.br/Despesa', 'despesas', 'ABO'],
 ['http://esic.vespasiano.mg.gov.br/FolhaPagamento', 'pessoal', 'ABO'],
 ['http://esic.vespasiano.mg.gov.br/FolhaPagamento', 'pessoal', 'ABO'],
 ['http://esic.vespasiano.mg.gov.br/Receita', 'receitas', 'ABO'],
 ['http://transparencia.virginia.mg.gov.br/Receita', 'receitas', 'ABO'],
 ['https://freigaspar.mg.gov.br/transparencia/licitacoes/detalhes/2021/12/110','licitacoes','Template 2'],
 ['https://freigaspar.mg.gov.br/transparencia/licitacoes','licitacoes','Template 2'],
 ['https://freigaspar.mg.gov.br/transparencia/pagamentos/detalhes/2022/04/119','pagamentos','Template 2'],
 ['https://freigaspar.mg.gov.br/transparencia/pagamentos','pagamentos','Template 2'],
 ['http://www.adpmnet.com.br/index.php?option=com_contpubl&submenu=0&cnpj=18125146000129&idreg=&tpformtab=jos_coputctr&tpformpdf=6&tpform=2&idorg=104&dsorg=Prefeitura%20Municipal%20de%20Buritis&dsufe=Estado%20de%20Minas%20Gerais&brasao=P310930.GIF&titulo=Contratos%20e%20seus%20Aditivos&idtpc=0&nome_mat=1&nao_proventos=0&nao_descontos=0','contratos', 'ADPM'],
 ['http://www.adpmnet.com.br/index.php?option=com_contpubl&submenu=0&cnpj=18125146000129&idreg=&tpformtab=jos_coputctr&tpformpdf=6&tpform=2&idorg=104&dsorg=Prefeitura%20Municipal%20de%20Serro&dsufe=Estado%20de%20Minas%20Gerais&brasao=P310930.GIF&titulo=Contratos%20e%20seus%20Aditivos&idtpc=0&nome_mat=1&nao_proventos=0&nao_descontos=0','contratos','ADPM'],
 ['http://www.adpmnet.com.br/index.php?option=com_contpubl&submenu=1&brasao=P310930.GIF&dsorg=Prefeitura%20Municipal%20de%20Buritis&cnpj=18125146000129&tpformpdf=70&ano=2022&mes=4&idorg=104&titulo=Pagamentos%20de%20Restos%20a%20Pagar&dsufe=Estado%20de%20Minas%20Gerais&nome_mat=1&nao_proventos=0&nao_descontos=0&xts=0','pagamentos', 'ADPM'],
 ['http://www.adpmnet.com.br/index.php?option=com_contpubl&submenu=1&brasao=P310930.GIF&dsorg=Prefeitura%20Municipal%20de%20Buritis&cnpj=18125146000129&tpformpdf=70&ano=2020&mes=4&idorg=104&titulo=Pagamentos%20de%20Restos%20a%20Pagar&dsufe=Estado%20de%20Minas%20Gerais&nome_mat=1&nao_proventos=0&nao_descontos=0&xts=0','pagamentos','ADPM'],
 ['http://www.adpmnet.com.br/index.php?option=com_contpubl&xts=0&idorg=104&ano=2020&mes=3&dsorg=Prefeitura%20Municipal%20de%20Buritis&dsufe=Estado%20de%20Minas%20Gerais&brasao=P310930.GIF&titulo=Despesas&tpformpdf=55&contraste=true','despesas', 'ADPM'],
 ['http://www.adpmnet.com.br/index.php?option=com_contpubl&xts=0&idorg=104&ano=2021&mes=7&dsorg=Prefeitura%20Municipal%20de%20Buritis&dsufe=Estado%20de%20Minas%20Gerais&brasao=P310930.GIF&titulo=Despesas&tpformpdf=55&contraste=true','despesas','ADPM'],
 ['http://www.adpmnet.com.br/index.php?option=com_contpubl&submenu=1&brasao=P310930.GIF&dsorg=Prefeitura%20Municipal%20de%20Buritis&cnpj=18125146000129&tpformpdf=51&ano=2022&mes=2&idorg=104&titulo=Remunera%C3%A7%C3%A3o%20Mensal%20dos%20Servidores%20P%C3%BAblicos&dsufe=Estado%20de%20Minas%20Gerais&nome_mat=1&nao_proventos=0&nao_descontos=0&xts=0','pessoal','ADPM'],
 ['http://www.adpmnet.com.br/index.php?option=com_contpubl&submenu=1&brasao=P310930.GIF&dsorg=Prefeitura%20Municipal%20de%20Buritis&cnpj=18125146000129&tpformpdf=51&ano=2017&mes=10&idorg=104&titulo=Remunera%C3%A7%C3%A3o%20Mensal%20dos%20Servidores%20P%C3%BAblicos&dsufe=Estado%20de%20Minas%20Gerais&nome_mat=1&nao_proventos=0&nao_descontos=0&xts=0','pessoal','ADPM'],
 ['http://www.adpmnet.com.br/index.php?option=com_contpubl&submenu=1&brasao=P310930.GIF&dsorg=Prefeitura%20Municipal%20de%20Buritis&cnpj=18125146000129&tpformpdf=49&ano=2022&mes=1&idorg=104&titulo=Receitas&dsufe=Estado%20de%20Minas%20Gerais&nome_mat=1&nao_proventos=0&nao_descontos=0&xts=0','receitas', 'ADPM'],
 ['http://www.adpmnet.com.br/index.php?option=com_contpubl&submenu=1&brasao=P310930.GIF&dsorg=Prefeitura%20Municipal%20de%20Buritis&cnpj=18125146000129&tpformpdf=49&ano=2020&mes=11&idorg=104&titulo=Receitas&dsufe=Estado%20de%20Minas%20Gerais&nome_mat=1&nao_proventos=0&nao_descontos=0&xts=0','receitas','ADPM']]

In [20]:
x_train = np.array([preProcessing(instance[0], instance[2]) for instance in dataset])

In [21]:
y_train_int = [label_to_int(instance[1]) for instance in dataset]
y_train = np_utils.to_categorical(y_train_int, NUMBER_OF_LABELS)
y_train

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0.

## Criação do modelo

In [22]:
def createModel():

    # Sequential layer
    model = Sequential()

    # fully connected layer
    model.add(Dense(55, activation='relu', input_shape=(NUMBER_OF_ATRIBUTES,)))
    model.add(Dense(NUMBER_OF_LABELS, activation='softmax'))

    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


In [23]:
model = createModel()

2022-08-18 01:24:43.045667: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-18 01:24:43.063679: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-18 01:24:43.068680: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (arthur): /proc/driver/nvidia/version does not exist
2022-08-18 01:24:43.148152: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 55)                5335      
                                                                 
 dense_1 (Dense)             (None, 6)                 336       
                                                                 
Total params: 5,671
Trainable params: 5,671
Non-trainable params: 0
_________________________________________________________________


In [24]:
response = model.fit(x_train, y_train, batch_size=16, epochs=25, verbose=1)

Epoch 1/25
3/3 [==============================] - 1s 11ms/step - loss: 23.3162 - accuracy: 0.2083
Epoch 2/25
3/3 [==============================] - 0s 5ms/step - loss: 17.0858 - accuracy: 0.2500
Epoch 3/25
3/3 [==============================] - 0s 5ms/step - loss: 10.8149 - accuracy: 0.3125
Epoch 4/25
3/3 [==============================] - 0s 5ms/step - loss: 6.5188 - accuracy: 0.3958
Epoch 5/25
3/3 [==============================] - 0s 6ms/step - loss: 4.6750 - accuracy: 0.5833
Epoch 6/25
3/3 [==============================] - 0s 6ms/step - loss: 3.7683 - accuracy: 0.6875
Epoch 7/25
3/3 [==============================] - 0s 6ms/step - loss: 2.8864 - accuracy: 0.6875
Epoch 8/25
3/3 [==============================] - 0s 6ms/step - loss: 2.0602 - accuracy: 0.7292
Epoch 9/25
3/3 [==============================] - 0s 6ms/step - loss: 1.4583 - accuracy: 0.7500
Epoch 10/25
3/3 [==============================] - 0s 5ms/step - loss: 0.9694 - accuracy: 0.8750
Epoch 11/25
3/3 [==================

## Testes de classificação

In [25]:
def classificarPagina(url, template):
    
    v1 = np.array([preProcessing(url, template)])
    probs = model.predict(v1)     
    idx = np.argmax(probs)
                   
    return int_to_label(idx)

In [27]:
classificarPagina("https://www.municipalnet.com.br/index/?apl=contas-publicas?uid=congonhal", "Municipal Net")

1/1 [==============================] - 0s 140ms/step


'despesas'

In [28]:
classificarPagina("https://www.municipalnet.com.br/tablicitacoes/?uid=albertina", "Municipal Net")

1/1 [==============================] - 0s 27ms/step


'licitacoes'

In [29]:
classificarPagina("https://pm-fervedouro.publicacao.siplanweb.com.br/contas-publicas/documento/105", "Siplanweb")

1/1 [==============================] - 0s 32ms/step


'despesas'

In [30]:
classificarPagina("https://pm-fervedouro.publicacao.siplanweb.com.br/restos-pagar", "Siplanweb")

1/1 [==============================] - 0s 26ms/step


'pagamentos'

In [31]:
classificarPagina("https://transparencia.teofilootoni.mg.gov.br/portalcidadao/#78c3e513dd43cb27d8a3e2f376196ffc656d7ea577b2c6fb35304957eca724c8880%C5%A6081606106eb8ab9bda7122e2f6d1dafa52a9147ca20f98f2111ce7722db81eb4b91157d05b0213d61a5f938482f6088effca1e34709dbe4ab78fdfc1757d92bb283bc08ad0ef86196179064665fa8c82e539fddca1396f40ae30893fb3031f99cc1818218c9ff2c4276e071ecb24b9b748f695e4794e0afb3fc709ec1cd2b7dcd635e57a223e1b5b5974c9de634b725378509390a6e7b8fce5f80469cd5eab92c1c3b3cc71b58774ff8c9155093fc03492de9b609a05", "GRP")

1/1 [==============================] - 0s 25ms/step


'pessoal'

In [32]:
classificarPagina("https://www.municipalnet.com.br/despesa_pessoal/", "Municipal Net")

1/1 [==============================] - 0s 41ms/step


'pessoal'

In [33]:
classificarPagina("https://www.municipalnet.com.br/folhadepagamento/", "Municipal Net")

1/1 [==============================] - 0s 25ms/step


'pessoal'

In [34]:
classificarPagina("http://servicos.betim.mg.gov.br/appsgi/servlet/wmportaltranspsemca?..%2F..%2Fappares%2Fservlet%2Fwmcontratotransparencia", "Municipal Net")

1/1 [==============================] - 0s 27ms/step


'contratos'